# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最终可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: ** 分类问题。
最终的结果是希望对学生分为两类，即不给予帮助也能正常毕业、以及不给予帮助不能正常毕业的。

这里，能够正常毕业和不能正常毕业的学生应该有一些共同的特性，而这些特性可能更多的是一些没有真正数学意义的分类数据(比如住的进或远)；而类似缺席次数这样的数据，也可以通过加权和阈值等去处理。

总的来说，我们希望最终是为了通过各种条件的组合判断把学生分为两类，因而它是一个分类问题。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [12]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv('student-data.csv')
print "Student data read successfully!"

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [13]:
# TODO： 计算学生的数量
n_students = len(student_data)

# TODO： 计算特征数量
n_features = len(student_data.columns)  - 1 # 数据框是包含了“label”变量的orz

# TODO： 计算通过的学生数
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO： 计算未通过的学生数
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO： 计算通过率
grad_rate = n_passed * 100 / n_students

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.00%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [14]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [15]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # 收集转换后的列
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
# print X_all

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约76%），使用95个数据点作为测试集（约24%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [16]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.model_selection import train_test_split

# TODO：设置训练集的数量
num_train = 300.0

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train


# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test / X_all.shape[0] , random_state = 3)


# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **
去掉KNN。虽然KNN训练用时短，但用于预测耗时长。而显然，为了能够在未来十年让毕业率达到95%，意味着我们需要对所有新入学的学生数据进行预测，节约成本上不符。

去掉逻辑回归。在特征空间较大时，逻辑回归的性能较差，且容易欠拟合。

去掉随机梯度下降。因为容易找到局部最优解而忽略掉全局最优解（虽然可以用退火处理，但个人尝试了一下，随机性有点高

~~去掉向量机。计算资源消耗大。~~ 我实际跑了下，发现向量机效果居然挺不错的…… checked

决策树 checked

集成算法之随机森林 checked

贝叶斯 checked

贝叶斯：
- 应用场景：[垃圾邮件分类](https://wshen.net/paper/Zheng_JNPU2010.pdf)，对于某些特定的发件人或者邮件出现某些特定的词汇时，可以判断出该邮件是垃圾邮件的概率大小，从而完成分类
- 优势：模型可以较为简单，速度快，对缺失数据不敏感，适合做文本处理
- 缺点：需要知道先验概率，对于一些较为复杂的数据（多维度数据——本项目就是）或属性之间有关联时效果差
- 选择原因：维度多，贝叶斯实际上并不是很适合这个题目……不过胜在模型简单速度快，所以虽然只让选三个模型，但我依然把它作为备选先丢进来

PS:虽然代码中没表现出来，实际上我在来回尝试中还是运行了它。。。

决策树：
- 应用场景：数据挖掘，如[这篇](http://pa.cm1911.com/Images/ResearchResources/基于CART决策树数据挖掘算法的应用研究.pdf)，通过类似于if_else的方式层层推进，推导出原因与结果之间的关系
- 优点：学习和分类速度都较快，可读性好，具有描述性，有助于人工分析，适合处理高维度数据。
- 缺点：容易过拟合；对于样本数量不一致的数据，信息增益的结果偏向于那些具有更多数值的特征。
- 选择原因：额，项目本身就很像一个数据挖掘的东东……

随机森林：
- 应有场景：[垃圾网页检测](http://xueshu.baidu.com/s?wd=paperuri%3A%2855781c748842dfaf066412f7cdeeb18b%29&filter=sc_long_sign&tn=SE_xueshusource_2kduw22v&sc_vurl=http%3A%2F%2Fwww.cqvip.com%2FQK%2F94832X%2F201603%2F668074251.html&ie=utf-8&sc_us=7590331524274648441)，其本身是随机构建多棵决策树，然后对对样本做分类，综合这些树的选择结果来作为最终决策结果。
- 优势：适合处理高维度数据，能有效的减少过拟合的问题
- 缺点：噪音较大的分类或回归问题上会过拟合；
- 选择原因：算是对决策树的一个补充。

向量机：
- 应用场景：[基于支持向量机与无监督聚类相结合的中文网页分类器](http://xueshu.baidu.com/s?wd=paperuri%3A%28aaa3818208e1182f2153c9da43e4ebe4%29&filter=sc_long_sign&tn=SE_xueshusource_2kduw22v&sc_vurl=http%3A%2F%2Fwww.cqvip.com%2FMain%2FDetail.aspx%3Fid%3D4839873&ie=utf-8&sc_us=18311622529853303960)，在由维度属性构建起的空间中，构建一个超维度平面，将数据一分为二，在分开数据的超平面的两边建有两个互相平行的超平面，分隔超平面使两个平行超平面的距离最大化。通过使方差最小以达到分类最优。
- 优势：最终决策函数只由少数的支持向量所确定，计算的复杂性取决于支持向量的数目，而不是样本空间的维数，这在某种意义上避免了“维数灾难”；同时也因为决定边界的是少量数据，可以剔除冗余数据，具有较好的鲁棒性。适合小规模训练样本
- 缺点：要对数据做二次规划，因而速度慢，大规模数据要消耗较多资源；经典向量机只提供二分类算法，无法做多分类处理。
- 选择原因：数据规模较小，同时也是二分类（合格不合格/是否能毕业）问题

PS:我本来以为随机森林会比较快的，所以最初是把向量机替换掉了的，但实际上随机森林运行时间比向量机要慢很多……看来各类文档文章说的速度快慢也都只是相对性的。

### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [17]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [39]:
# TODO：从sklearn中引入三个监督学习模型
from sklearn.ensemble import RandomForestClassifier 
# from sklearn.ensemble import AdaBoostClassifier # 训练时间长，效果一般
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC # 目前看这个居然最好

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

random_num = 5

# TODO：初始化三个模型
clf_A = DecisionTreeClassifier(random_state = random_num,min_samples_split = 3)
clf_B = RandomForestClassifier(random_state = random_num )
clf_C = SVC(random_state = random_num)

"""
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test / X_all.shape[0] ,
                                                   random_state = 3)
# """

# TODO：设置训练集大小
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print '---'
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print '---'
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

print "\n-----------------"

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print '---'
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print '---'
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

print "\n-----------------"

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print '---'
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print '---'
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.6457.
---
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0017 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.9960.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7794.
---
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0020 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.9922.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7746.

-----------------
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0357 seconds
Made predictions in 0.0056 seconds.
F1 score for training set: 0.9922.
Made predictions in 0.0072 seconds.
F1 score for test set: 0.8344.
---
T

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - 决策树 **  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0017           |       0.0004           |   1              |      0.6457     |
| 200               |        0.0017           |       0.0003           |   0.9960         |      0.7794     |
| 300               |        0.0020           |       0.0005           |   0.9922         |      0.7746     |

** 分类器 2 - 随机森林 **  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0357           |       0.0072           |   0.9922         |    0.8344       |
| 200               |        0.0265           |       0.0058           |   0.9881         |    0.8310       |
| 300               |        0.0332           |       0.0057           |   0.9820         |    0.7801       |

** 分类器 3 - 支持向量机 **  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0012           |       0.0008           |    0.9028        |    0.8408       |
| 200               |        0.0034           |       0.0017           |    0.8571        |    0.8462       |
| 300               |        0.0071           |       0.0020           |    0.8739        |    0.8354       |

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **
从时间成本上(因为是基于运行时间来计费，经济成本与时间成本挂钩)，随机森林最差，同时其效果也并不出众，排除。

虽然决策树的速度最快，但从运行结果上来看，其效果并不算很好。因为目前的不合格率是33%，而为了能够让合格率达到95%以上，我们需要对这不合格的人做干预，那么其预测准确度应该是在 .27 / .33 ≈ 81.82% 以上（这里我假设所有被干预的学生都能成功毕业，忽略了本可以毕业但也被错误预测的那部分同学），因而决策树的效果并不理想，也排除。

综合预测效果和时间（经济）条件，支持向量机能有效达到提高升学率的目的、并且开支也较为合理。

-------------

***第二次作答：***
```
补充：加上random_state后发现随着修改random_state的值，决策树和随机森林的预测结果偶尔会出现非常好的情况，但这种情况并不稳定，因此认为二者并不够健壮；相比之下向量机一直都保持在一个很好的预测准确度上，因此依然还是认为选择使用向量机。
```

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **
假设一张白纸上摆满学生，能通过最终考试的学生扎堆在中间，无法通过最终考试的学生，或因为家庭环境差(famrel)或因为过去考试失败次数较多(failures)，分布在白纸的边角。这个模型则是一把刀，可以把边角切下来、从而达到分辨学生是否能够pass的目的。我们把白纸横向对折再纵向对折，然后只要切一刀，就能把四个角都切下来。

当然，无法pass的学生可能并不仅仅是再四个角，所以我们需要把纸张多对折几次、卷成一个圆柱或者一个锥形、叠成一个纸飞机甚至是千纸鹤，然后才能用模型一刀切开。

同时，就像学生在课桌上画三八线要画在桌子正中央、从而给予学生足够的活动区间免得日常伸个懒腰就不小心越界了一样，模型切的这一刀也要切在能pass和不能pass的学生中间位置，这样某些压线的学生就能有一定的容错区间。

模型训练过程就是这个寻找折纸方法和下刀切的位置的过程。

而预测，就只要按照正常流程把被预测的学生数据放在纸上，然后看他究竟是落在了被切开的纸张的哪一侧了。

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [43]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from sklearn.model_selection import GridSearchCV,ShuffleSplit
from sklearn.metrics import make_scorer,f1_score

# TODO：创建你希望调整的参数列表
'''
parameters = { 'kernel':('linear','rbf', 'sigmoid','poly' ),
              'max_iter':[-1,50,100,200],'C':[.5,.8,1.0,2.0],
              'gamma':('auto',0.1,0.2,0.05)}
# '''
# 这个parameters是老师推荐的参数……
parameters = [{'C': [0.1, 0.5, 1, 10], 'kernel': ['linear']},
              {'C': [0.1, 0.5, 1, 10],'degree':[2, 3], 'gamma': [0.5, 0.1, 0.01], 'kernel': ['poly']},
              {'C': [0.1, 0.5, 1, 10], 'gamma': [0.5, 0.1, 0.01], 'kernel': ['rbf']},
              {'C': [0.1, 0.5, 1, 10, 100]}]

# """
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test / X_all.shape[0] , 
                                                    random_state = 7)
# """


# TODO：初始化分类器
clf = SVC()

cv_sets = ShuffleSplit(n_splits = 20, test_size = 0.2 ,random_state = 1)

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score,pos_label = 'yes')

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf,parameters,scoring = f1_scorer, cv = cv_sets)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数


grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print clf
print grid_obj.best_score_
print grid_obj.best_params_


train_predict(clf_C, X_train, y_train, X_test, y_test)

Made predictions in 0.0057 seconds.
Tuned model has a training F1 score of 0.9774.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8375.
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.807453524009
{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}
Training a SVC using a training set size of 300. . .
Trained model in 0.0059 seconds
Made predictions in 0.0048 seconds.
F1 score for training set: 0.8739.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.8354.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **
在对X,y分割的random_state = 3的情况下，最终模型的训练F1:0.9734,测试F1:0.8289

这个值相对没有调整过参数的模型，训练F1高了很多，但测试F1低得有限（这里降低了，应该是与cv折的random_state有关），可以说稳定性更高，或许存在一点点过拟合。

为了对这个问题做更进一步的确认，我修改了random_state（因为最初我没用gamma参数时，设置的random_state = 1，然后发现，修改向量机参数居然并不会影响F1的结果值，训练F1都是0.8584，测试结果都是0.8462）

|random_state值|训练F1|预测F1|调参后训练F1|调参后预测F1|最优调参|
|:-:|:-:|:-:|:-:|:-:|:-:|
|1|0.8584|0.8462|0.9000|0.8462|{'kernel': 'rbf', 'C': 0.8, 'max_iter': 200, 'gamma': 0.05}|
|2|0.8504|0.8267|0.9852|0.8333|{'kernel': 'rbf', 'C': 1.0, 'max_iter': -1, 'gamma': 0.1}|
|3|0.8529|0.8356|0.9734|0.8289|{'kernel': 'rbf', 'C': 1.0, 'max_iter': -1, 'gamma': 0.1}|
|4|0.8619|0.7639|0.8819|0.7671|{'kernel': 'rbf', 'C': 0.8, 'max_iter': -1, 'gamma': 0.05}|
|5|0.8764|0.7660|0.9760|0.7922|{'kernel': 'rbf', 'C': 1.0, 'max_iter': -1, 'gamma': 0.1}|
|6|0.8529|0.8456|0.8403|0.8477|{'kernel': 'rbf', 'C': 0.8, 'max_iter': -1, 'gamma': 'auto'}|
|7|0.8739|0.8354|0.9774|0.8375|{'kernel': 'rbf', 'C': 1.0, 'max_iter': -1, 'gamma': 0.1}|
等等。

如上，虽然调参确实能稍稍提高一点预测F1，但基本上都很有限，而且并不能保证新的调参能够很好适应其他random_state值的情况，因而我认为这里调参并无太大的必要。

--------

***第二次作答：***

>如果可能的话，调整更多的参数进行尝试。

>parameters = [{'C': [0.1, 0.5, 1, 10], 'kernel': ['linear']},
  {'C': [0.1, 0.5, 1, 10],'degree':[2, 3], 'gamma': [0.5, 0.1, 0.01], 'kernel': ['poly']},
  {'C': [0.1, 0.5, 1, 10], 'gamma': [0.5, 0.1, 0.01], 'kernel': ['rbf']}]

>其实C的取值还是很影响我们的模型的，如下图所示，C的取值范围也较大。


```
上述为老师的上一次的建议，然后这里我把老师提供的parameters放进了代码中，结果训练F1为0.8739，预测F1为0.8354，与之前没调参时对X_train_300, y_train_300的处理结果一样。
而最优参数为  {'kernel': 'rbf', 'C': 1, 'gamma': 0.1}
所以依然还是认为没有必要调参。
PS：实际上之前尝试的参数不仅仅是我第一次罗列在代码上的那些……我当时试了更多……然后发现因为组合项太多了，运行起来把我内存太占内存影响其他工作了，所以后续做其他验证的时候去除了看起来没啥影响的一些参数，只保留上面几个……
```

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  